In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


In [2]:
from keras.applications import MobileNet

size = 224

mobile_net = MobileNet(weights="imagenet", include_top=False, input_shape=(size, size, 3))

for layer in mobile_net.layers:
    layer.trainable = False

for i, layer in enumerate(mobile_net.layers):
    print("{}\t{:18}\t{!s}".format(i, layer.__class__.__name__, layer.trainable))

Using TensorFlow backend.
0	InputLayer        	False
1	ZeroPadding2D     	False
2	Conv2D            	False
3	BatchNormalization	False
4	ReLU              	False
5	DepthwiseConv2D   	False
6	BatchNormalization	False
7	ReLU              	False
8	Conv2D            	False
9	BatchNormalization	False
10	ReLU              	False
11	ZeroPadding2D     	False
12	DepthwiseConv2D   	False
13	BatchNormalization	False
14	ReLU              	False
15	Conv2D            	False
16	BatchNormalization	False
17	ReLU              	False
18	DepthwiseConv2D   	False
19	BatchNormalization	False
20	ReLU              	False
21	Conv2D            	False
22	BatchNormalization	False
23	ReLU              	False
24	ZeroPadding2D     	False
25	DepthwiseConv2D   	False
26	BatchNormalization	False
27	ReLU              	False
28	Conv2D            	False
29	BatchNormalization	False
30	ReLU              	False
31	DepthwiseConv2D   	False
32	BatchNormalization	False
33	ReLU              	False
34	Conv2D            	False
35	B

In [3]:
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

full_model = mobile_net.output
full_model = GlobalAveragePooling2D()(full_model)
full_model = Dense(1024, activation="relu")(full_model)
full_model = Dense(1024, activation="relu")(full_model)
full_model = Dense(512, activation="relu")(full_model)
# total classes 10
full_model = Dense(10, activation="softmax")(full_model)

model = Model(inputs=mobile_net.inputs, outputs=full_model)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = "./data/monkeys/train"
test_data_dir = "./data/monkeys/validation"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(size, size),
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(size,size),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [5]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("./monkey_breed_v1.h5", monitor="val_loss", mode="min", save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, restore_best_weights=True)

callbacks = [checkpoint, earlystop]

model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=0.001), metrics=["accuracy"])

nb_train = 1098
nb_test = 272
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train//batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_generator,
    validation_steps=nb_test//batch_size
)

Epoch 1/5
68/68 [==============================] - 65s 963ms/step - loss: 1.9140 - accuracy: 0.4719 - val_loss: 0.4457 - val_accuracy: 0.8182

Epoch 00001: val_loss improved from inf to 0.44575, saving model to ./monkey_breed_v1.h5
Epoch 2/5
68/68 [==============================] - 63s 933ms/step - loss: 0.7212 - accuracy: 0.7725 - val_loss: 0.5655 - val_accuracy: 0.8516

Epoch 00002: val_loss did not improve from 0.44575
Epoch 3/5
68/68 [==============================] - 62s 910ms/step - loss: 0.5629 - accuracy: 0.8368 - val_loss: 0.2742 - val_accuracy: 0.7988

Epoch 00003: val_loss improved from 0.44575 to 0.27421, saving model to ./monkey_breed_v1.h5
Epoch 4/5
68/68 [==============================] - 59s 867ms/step - loss: 0.4078 - accuracy: 0.8804 - val_loss: 0.0736 - val_accuracy: 0.9219

Epoch 00004: val_loss improved from 0.27421 to 0.07364, saving model to ./monkey_breed_v1.h5
Epoch 5/5
68/68 [==============================] - 56s 824ms/step - loss: 0.3888 - accuracy: 0.8914 - 